In [1]:
OPENAI_API_KEY=""

In [2]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

In [3]:
MAX_TOKEN = 6000
MAX_CHAT = 10
SCENARIO_ID = 1
PERSONA = 'submissive_people'
AGENT_NAME = 'sparky'

In [4]:
from enum import Enum
from pydantic import BaseModel

class UserAction(str, Enum):
    say = "say"
    leave = "leave"

class UserResponse(BaseModel):
    action: UserAction
    answer: str

In [29]:
import json
class Person:
    def __init__(self, scenarioID: int, persona: str):

        scenarioFile = open(f'./user/{AGENT_NAME}/scenario{scenarioID}.txt', 'r')
        scenario = scenarioFile.read()
        scenarioFile.close()

        personaFile = open(f'./persona/{persona}.txt', 'r')
        persona = personaFile.read()
        personaFile.close()
        example = '''
                    Example1:
                    Input: Are you looking for relaxation techniques or some fun trivia games to de-stress? Let me help you find the best fit!
                    Output: {action: say, answer: 'I am looking for relaxation techniques'}
                    Example2:
                    Input: There's a wonderful bear named Bruno who specializes in relaxation techniques. Would you like to meet him for some calming mindfulness tips?
                    Output: {action: leave, answer: 'Yes, meeting Bruno sounds lovely! I would love to get some calming mindfulness tips from him.'}
                  '''
        systemPrompt = f'You are a user talking to AI APP which can help you deal with your problem during break time. \
                            This is your persona: {persona}\
                            Please play the role according to the scenario: {scenario}\
                            Use Action → Answer structure for responses.\
                            Available Actions:\
                            1. say: respond base on persona and scenario\
                            2. leave: leave the chat when you think the conversation is over, no need to continue\
                            Examples:\n{example}'
        # print(systemPrompt)

        self.messages = [
            {'role': 'system', 'content': systemPrompt}, 
        ]
        self.leaveChat = False
    
    def say(self):
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=self.messages,
            response_format={
                'type': 'json_schema',
                'json_schema': 
                    {
                        "name":"whocares", 
                        "schema": UserResponse.model_json_schema()
                    }
            }
        )

        message = response.choices[0].message.content
        self.messages.append({'role': 'assistant', 'content': message})

        # str to dict
        message = json.loads(message)
        self.leaveChat = (message['action'] == 'leave')

        info = {
            'token': response.usage.total_tokens,
        }
        
        return message['answer'], info
    
    def listen(self, message: str):
        self.messages.append({'role': 'user', 'content': message})

        

In [30]:
test = Person(SCENARIO_ID, PERSONA)
ans = test.say()
print(ans)

('What does this app do?', {'token': 363})


In [7]:
class sparkyAction(str, Enum):
    call_bruno = "call_bruno"
    call_bizy = "call_bizy"
    ask_more = "ask_more"
    introduce_bruno = "introduce_bruno"
    introduce_bizy = "introduce_bizy"
    advise = "advise"

class AgentResponse(BaseModel):
    action: sparkyAction
    answer: str

In [8]:
import time
import json

class Animal:
    def __init__(self, name: str):

        self.agent_name = name
        agentFile = open(f'./agent/{name}.txt', 'r')
        agentPrompt = agentFile.read()
        agentFile.close()

        # create assistant and thread
        self.assistant = client.beta.assistants.create(
            name = self.agent_name,
            instructions = agentPrompt,
            model="gpt-4o-mini",
            response_format={
                'type': 'json_schema',
                'json_schema': 
                    {
                        "name":"whocares", 
                        "schema": AgentResponse.model_json_schema()
                    }
            }
        )
        self.thread = client.beta.threads.create()
        self.user_message = 'hello'

    def say(self):

        prompt = client.beta.threads.messages.create(
            thread_id = self.thread.id,
            role = "user",
            content = self.user_message
        )

        run = client.beta.threads.runs.create_and_poll(
            thread_id=self.thread.id,
            assistant_id=self.assistant.id,
        )

        while True:
            runData = client.beta.threads.runs.retrieve(
                thread_id=self.thread.id,
                run_id=run.id
            )

            if runData.status == 'completed': 
                response = client.beta.threads.messages.list(
                    thread_id=self.thread.id
                )
                message = json.loads(response.data[0].content[0].text.value)

                info = {
                    'token': runData.usage.total_tokens,
                    'action': message['action']
                }
                return message['answer'], info

            else:
                print("runData.status")
                time.sleep(2) 


    def listen(self, message: str):
        self.user_message = message
    

In [9]:
test = Animal(name = AGENT_NAME)
print(test.say())
test.listen('i feel tired')
print(test.say())

("Hello there, delightful friend! How can I help you today? I'm here to spread positivity!", {'token': 520, 'action': 'advise'})
("It sounds like you could use a little zen time! There's a bear named Bruno who specializes in relaxation. Would you like to meet him?", {'token': 570, 'action': 'introduce_bruno'})


In [10]:
from pydantic import BaseModel

class EvaluatorResponse(BaseModel):
    accuracy: int
    practicality: int

In [24]:
import json
class Evaluator():
    def __init__(self) -> None:

        systemPrompt = '''You are an evaluator. I will provide you with a user’s statement and an agent’s response.
                            You should evaluate the accuracy and practicality base on the scenario.
                            - Accuracy: Score from 0 to 10. This measures whether the model’s response appropriately addresses the user’s statement.
                            - Practicality: Score from 0 to 10. This evaluates whether the model’s suggestion is helpful to the user.
                        '''
        with open(f'./evaluator/{AGENT_NAME}/scenario{SCENARIO_ID}.txt', 'r') as file:
            systemPrompt += file.read()
        with open(f'./evaluator/examples.txt', 'r') as file:
            systemPrompt += file.read()

        self.messages = [
            {'role': 'system', 'content': systemPrompt}, 
        ]
        
    def evaluate(self, personMessage: str, animalMessage: str):
        self.messages.append({'role': 'user', 'content': f'User: {personMessage}\nAgent: {animalMessage}'})
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=self.messages,
            response_format={
                'type': 'json_schema',
                'json_schema': 
                    {
                        "name":"whocares", 
                        "schema": EvaluatorResponse.model_json_schema()
                    }
            }
        )

        message = json.loads(response.choices[0].message.content)
    
        return message['accuracy'], message['practicality']


In [28]:
test = Evaluator()
test.evaluate('What does this app do?', 'I do not know')

(0, 0)

In [13]:
import pandas as pd
class Report():
    def __init__(self) -> None:
        self.finish = True
        self.chatHistory = []

    def addHistory(self,chat):
        self.chatHistory.append(chat)
    
    def generateReport(self):
        df = pd.DataFrame(self.chatHistory)
        df.to_csv('report.csv', index=False)

In [19]:
person = Person(scenarioID = SCENARIO_ID, persona = PERSONA)
animal = Animal(name= AGENT_NAME)
evaluator = Evaluator()
report = Report()

totalToken = 0
totalChat = 0

while not person.leaveChat:
    # chat
    personMessage, personInfo = person.say()
    animal.listen(personMessage)
    animalMessage, animalInfo = animal.say()
    person.listen(animalMessage)

    print(f'User: {personMessage}\nAgent: {animalMessage}')
    print(f'user leave chat: {person.leaveChat}')

    # metrics
    accuracy, practicality = evaluator.evaluate(personMessage, animalMessage)
    print(f'Accuracy: {accuracy}, Practicality: {practicality}\n')
    

    report
    history = {
        'person_say': personMessage,
        'animal_action': animalInfo['action'],
        'animal_say': animalMessage,
        'animal_token': animalInfo['token'],
        'accuracy': accuracy,
        'practicality': practicality,
    }

    report.addHistory(history)
    totalChat += 1
    totalToken += animalInfo['token']

    if totalToken > MAX_TOKEN or totalChat > MAX_CHAT:
        report.finish = False
        break



User: What does this app do?
Agent: This app connects you with friendly animal specialists who can help you tackle stress, procrastination, or social challenges! Each animal has its own unique expertise, just like each tree has its own special shade!
user leave chat: False
Accuracy: 8, Practicality: 7

User: How can you help me?
Agent: What specific issue are you facing? Whether it's stress, procrastination, or making friends, I have some furry and buzzing friends ready to help!
user leave chat: False
Accuracy: 9, Practicality: 8

User: I'm facing some stress and would appreciate any help with that.
Agent: There's a wise bear named Bruno who specializes in mindfulness! Would you like to meet him to help reduce your stress?
user leave chat: False
Accuracy: 9, Practicality: 8

User: Yes, meeting Bruno sounds great! I would love to learn about mindfulness to help reduce my stress.
Agent: Awesome! Let me call Bruno right now. Get ready to unwind and discover the magic of mindfulness!
user 

In [20]:
report.generateReport()

In [21]:
report.chatHistory

[{'person_say': 'What does this app do?',
  'animal_action': 'advise',
  'animal_say': 'This app connects you with friendly animal specialists who can help you tackle stress, procrastination, or social challenges! Each animal has its own unique expertise, just like each tree has its own special shade!',
  'animal_token': 546,
  'accuracy': 8,
  'practicality': 7},
 {'person_say': 'How can you help me?',
  'animal_action': 'ask_more',
  'animal_say': "What specific issue are you facing? Whether it's stress, procrastination, or making friends, I have some furry and buzzing friends ready to help!",
  'animal_token': 598,
  'accuracy': 9,
  'practicality': 8},
 {'person_say': "I'm facing some stress and would appreciate any help with that.",
  'animal_action': 'introduce_bruno',
  'animal_say': "There's a wise bear named Bruno who specializes in mindfulness! Would you like to meet him to help reduce your stress?",
  'animal_token': 652,
  'accuracy': 9,
  'practicality': 8},
 {'person_say'

In [ ]:
print(totalToken, totalChat)